In [1]:
import pandas as pd

In [2]:
tripadv_review = pd.read_csv(r'C:\Users\Dell\Desktop\Hotel\Europa Plaza Hotel.csv', error_bad_lines=False);
tripadv_review

,r_points,r_text,place
0,40,Very nice people. They were very helpful. The ...,Europa Plaza Hotel
1,10,Do not stay at this hotel unless you like loud...,Europa Plaza Hotel
2,10,Don't waste money. Very rude receptionist . If...,Europa Plaza Hotel
3,20,Many things to report about this hotel. Unfort...,Europa Plaza Hotel
4,40,We stayed five nights in early October 2018. I...,Europa Plaza Hotel
...,...,...,...
68,30,"The Europa is a quiet, clean and friendly hote...",Europa Plaza Hotel
69,40,Stay in this hotel for 5 days in AugustVery fr...,Europa Plaza Hotel
70,50,A beautiful experience! Nice clean and large r...,Europa Plaza Hotel
71,40,"Not much I can say really, a nice clean, moder...",Europa Plaza Hotel


In [3]:
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sent = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [4]:
sent.polarity_scores(tripadv_review.iloc[0]['r_text'])

{'neg': 0.0, 'neu': 0.793, 'pos': 0.207, 'compound': 0.8548}

In [5]:
tripadv_review['scores'] = tripadv_review['r_text'].apply(lambda review:sent.polarity_scores(review))
tripadv_review['compound'] = tripadv_review['scores'].apply(lambda d:d['compound'])
tripadv_review['score'] = tripadv_review['compound'].apply(lambda score: 'pos' if score >=0 else 'neg')
tripadv_review.head()

,r_points,r_text,place,scores,compound,score
0,40,Very nice people. They were very helpful. The ...,Europa Plaza Hotel,"{'neg': 0.0, 'neu': 0.793, 'pos': 0.207, 'comp...",0.8548,pos
1,10,Do not stay at this hotel unless you like loud...,Europa Plaza Hotel,"{'neg': 0.151, 'neu': 0.778, 'pos': 0.071, 'co...",-0.6908,neg
2,10,Don't waste money. Very rude receptionist . If...,Europa Plaza Hotel,"{'neg': 0.197, 'neu': 0.663, 'pos': 0.141, 'co...",-0.7150,neg
3,20,Many things to report about this hotel. Unfort...,Europa Plaza Hotel,"{'neg': 0.081, 'neu': 0.902, 'pos': 0.017, 'co...",-0.7168,neg
4,40,We stayed five nights in early October 2018. I...,Europa Plaza Hotel,"{'neg': 0.0, 'neu': 0.831, 'pos': 0.169, 'comp...",0.7264,pos


In [9]:
tripadv_review.to_csv(r'C:\Users\Dell\Desktop\sentiment_out2.csv')

In [10]:
pos_rev=tripadv_review[(tripadv_review['score'] .str.contains('pos'))]
pos_rev.to_csv(r'C:\Users\Dell\Desktop\sentiment_out2_pos.csv')
neg_rev=tripadv_review[(tripadv_review['score'] .str.contains('neg'))]
neg_rev.to_csv(r'C:\Users\Dell\Desktop\sentiment_out2_neg.csv')

In [11]:
from pycaret.nlp import *

In [12]:
exp_nlp = setup(data = pos_rev, target = 'r_text',
                custom_stopwords = [ 'hotel', 'room', 'nice', 'good', 'excellent','restaurant', 'food'])

Description,Value
session_id,7437
Documents,60
Vocab Size,876
Custom Stopwords,True


In [13]:
#create an LDA model
lda_pos_custom = create_model('lda')
print(lda_pos_custom)

LdaModel(num_terms=876, num_topics=4, decay=0.5, chunksize=100)


In [14]:
apply_lda_pos = assign_model(lda_pos_custom)
apply_lda_pos

,r_points,r_text,place,scores,compound,score,Topic_0,Topic_1,Topic_2,Topic_3,Dominant_Topic,Perc_Dominant_Topic
0,40.0,people helpful breakfast get close door hear n...,Europa Plaza Hotel,"{'neg': 0.0, 'neu': 0.793, 'pos': 0.207, 'comp...",0.8548,pos,0.004648,0.004505,0.983936,0.006911,Topic 2,0.98
1,NaN,NaN,NaN,NaN,NaN,NaN,0.005151,0.004992,0.982198,0.007659,Topic 2,0.98
2,NaN,NaN,NaN,NaN,NaN,NaN,0.002761,0.002676,0.004482,0.990081,Topic 3,0.99
3,NaN,NaN,NaN,NaN,NaN,NaN,0.002761,0.002676,0.990457,0.004105,Topic 2,0.99
4,40.0,stay night early locate close city worth stay ...,Europa Plaza Hotel,"{'neg': 0.0, 'neu': 0.831, 'pos': 0.169, 'comp...",0.7264,pos,0.000273,0.000265,0.999056,0.000406,Topic 2,1.00
...,...,...,...,...,...,...,...,...,...,...,...,...
68,30.0,quiet clean friendly minute walk book website ...,Europa Plaza Hotel,"{'neg': 0.022, 'neu': 0.855, 'pos': 0.123, 'co...",0.8839,pos,NaN,NaN,NaN,NaN,NaN,NaN
69,40.0,stay day staff reception clean fresh towel fre...,Europa Plaza Hotel,"{'neg': 0.039, 'neu': 0.787, 'pos': 0.174, 'co...",0.8316,pos,NaN,NaN,NaN,NaN,NaN,NaN
70,50.0,beautiful experience clean large change everyd...,Europa Plaza Hotel,"{'neg': 0.024, 'neu': 0.637, 'pos': 0.339, 'co...",0.9708,pos,NaN,NaN,NaN,NaN,NaN,NaN
71,40.0,much say really clean modern aim mainly busine...,Europa Plaza Hotel,"{'neg': 0.022, 'neu': 0.746, 'pos': 0.232, 'co...",0.9863,pos,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
evaluate_model(lda_pos_custom)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Frequency Plot', 'freque…

In [16]:
plot_model(lda_pos_custom, plot ='topic_model')

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3     -0.083745 -0.056153       1        1  33.551624
2     -0.047369  0.085142       2        1  28.947657
1      0.063725 -0.076606       3        1  20.622318
0      0.067389  0.047617       4        1  16.878397, topic_info=      Term       Freq      Total Category  logprob  loglift
13    stay  20.000000  20.000000  Default  30.0000  30.0000
207    ask   9.000000   9.000000  Default  29.0000  29.0000
214    car   5.000000   5.000000  Default  28.0000  28.0000
391   seem   6.000000   6.000000  Default  27.0000  27.0000
189   hard   5.000000   5.000000  Default  26.0000  26.0000
..     ...        ...        ...      ...      ...      ...
346  value   2.024861   8.276683   Topic4  -5.1983   0.3712
207    ask   2.072322   9.571739   Topic4  -5.1751   0.2490
7    noise   2.016219   9.812334   Topic4  -5.2026   0.1967
160   well   2.032237  17.272554   Topic4  -5.1947  -0.3608
12   night   2.039966  25.350792   Topic4  -5.1909  -0.7407

[280 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
758       4  0.753562  abundant
732       3  0.722604  activity
275       1  0.298799  adequate
275       4  0.597597  adequate
49        1  0.827110       air
...     ...       ...       ...
46        1  0.233665      work
46        2  0.350497      work
46        3  0.233665      work
46        4  0.116832      work
15        2  0.688042     worth

[404 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 3, 2, 1])

In [17]:
tune_lda_positive = tune_model(model='lda', supervised_target=None, estimator='xgboost')

IntProgress(value=0, description='Processing: ', max=25)

Output()

Best Model: Latent Dirichlet Allocation | # Topics: 64 | Coherence: 0.5399


16 topics selected

## NEGATIVE REVIEWS

In [22]:
exp_nlp_neg = setup(data = neg_rev, target = 'r_text',
                custom_stopwords = [ 'hotel', 'room', 'nice', 'good','restaurant', 'food','bad'])

Description,Value
session_id,4932
Documents,13
Vocab Size,334
Custom Stopwords,True


In [23]:
#create an LDA model
lda_neg_custom = create_model('lda')
print(lda_neg_custom)

LdaModel(num_terms=334, num_topics=4, decay=0.5, chunksize=100)


In [25]:
apply_lda_neg = assign_model(lda_neg_custom)
apply_lda_neg

,r_points,r_text,place,scores,compound,score,Topic_0,Topic_1,Topic_2,Topic_3,Dominant_Topic,Perc_Dominant_Topic
0,NaN,NaN,NaN,NaN,NaN,NaN,0.993659,0.002836,0.002462,0.001044,Topic 0,0.99
1,10.0,stay loud music noisy traffic motorcycle scree...,Europa Plaza Hotel,"{'neg': 0.151, 'neu': 0.778, 'pos': 0.071, 'co...",-0.6908,neg,0.004516,0.989022,0.004537,0.001924,Topic 1,0.99
2,10.0,receptionist want experience go head try peopl...,Europa Plaza Hotel,"{'neg': 0.197, 'neu': 0.663, 'pos': 0.141, 'co...",-0.7150,neg,0.001104,0.001277,0.997149,0.000470,Topic 2,1.00
3,20.0,many thing report unfortunately late night arr...,Europa Plaza Hotel,"{'neg': 0.081, 'neu': 0.902, 'pos': 0.017, 'co...",-0.7168,neg,0.005255,0.006081,0.986425,0.002239,Topic 2,0.99
4,NaN,NaN,NaN,NaN,NaN,NaN,0.000617,0.998501,0.000619,0.000263,Topic 1,1.00
5,10.0,book conference suggest dirty carpet remaining...,Europa Plaza Hotel,"{'neg': 0.083, 'neu': 0.917, 'pos': 0.0, 'comp...",-0.4404,neg,0.991778,0.003676,0.003192,0.001354,Topic 0,0.99
6,NaN,NaN,NaN,NaN,NaN,NaN,0.006282,0.984730,0.006311,0.002677,Topic 1,0.98
7,NaN,NaN,NaN,NaN,NaN,NaN,0.001522,0.001761,0.996068,0.000648,Topic 2,1.00
8,NaN,NaN,NaN,NaN,NaN,NaN,0.994108,0.002635,0.002287,0.000970,Topic 0,0.99
9,NaN,NaN,NaN,NaN,NaN,NaN,0.004516,0.989022,0.004537,0.001924,Topic 1,0.99


In [26]:
evaluate_model(lda_neg_custom)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Frequency Plot', 'freque…

In [29]:
tune_lda_negative = tune_model(model='lda', supervised_target=None, estimator='xgboost')

IntProgress(value=0, description='Processing: ', max=25)

Output()

Best Model: Latent Dirichlet Allocation | # Topics: 16 | Coherence: 0.6922


In [30]:
lda_neg_tuned = create_model('lda', num_topics=2)
print(lda_neg_tuned)

LdaModel(num_terms=334, num_topics=2, decay=0.5, chunksize=100)


In [31]:
apply_tuned_neg = assign_model(lda_neg_tuned)
apply_tuned_neg

,r_points,r_text,place,scores,compound,score,Topic_0,Topic_1,Dominant_Topic,Perc_Dominant_Topic
0,NaN,NaN,NaN,NaN,NaN,NaN,0.997523,0.002476,Topic 0,1.00
1,10.0,stay loud music noisy traffic motorcycle scree...,Europa Plaza Hotel,"{'neg': 0.151, 'neu': 0.778, 'pos': 0.071, 'co...",-0.6908,neg,0.004254,0.995746,Topic 1,1.00
2,10.0,receptionist want experience go head try peopl...,Europa Plaza Hotel,"{'neg': 0.197, 'neu': 0.663, 'pos': 0.141, 'co...",-0.7150,neg,0.998887,0.001113,Topic 0,1.00
3,20.0,many thing report unfortunately late night arr...,Europa Plaza Hotel,"{'neg': 0.081, 'neu': 0.902, 'pos': 0.017, 'co...",-0.7168,neg,0.004955,0.995045,Topic 1,1.00
4,NaN,NaN,NaN,NaN,NaN,NaN,0.000577,0.999423,Topic 1,1.00
5,10.0,book conference suggest dirty carpet remaining...,Europa Plaza Hotel,"{'neg': 0.083, 'neu': 0.917, 'pos': 0.0, 'comp...",-0.4404,neg,0.002985,0.997015,Topic 1,1.00
6,NaN,NaN,NaN,NaN,NaN,NaN,0.005934,0.994066,Topic 1,0.99
7,NaN,NaN,NaN,NaN,NaN,NaN,0.998464,0.001536,Topic 0,1.00
8,NaN,NaN,NaN,NaN,NaN,NaN,0.997700,0.002300,Topic 0,1.00
9,NaN,NaN,NaN,NaN,NaN,NaN,0.004254,0.995746,Topic 1,1.00


In [32]:
evaluate_model(lda_neg_tuned)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Frequency Plot', 'freque…

In [33]:
plot_model(lda_neg_tuned, plot ='topic_model')

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.019941  0.062949       1        1  60.431934
1      0.019941 -0.062949       2        1  39.568069, topic_info=             Term      Freq     Total Category  logprob  loglift
41      breakfast  4.000000  4.000000  Default  30.0000  30.0000
35           want  2.000000  2.000000  Default  29.0000  29.0000
107           bed  2.000000  2.000000  Default  28.0000  28.0000
149        police  2.000000  2.000000  Default  27.0000  27.0000
192        shower  1.000000  1.000000  Default  26.0000  26.0000
..            ...       ...       ...      ...      ...      ...
65          noise  1.314459  2.891796   Topic2  -4.9522   0.1387
32   receptionist  0.793042  1.734195   Topic2  -5.4575   0.1447
147         offer  0.794710  3.624288   Topic2  -5.4554  -0.5903
8            half  0.794186  2.364110   Topic2  -5.4560  -0.1637
187      business  0.794169  2.364113   Topic2  -5.4560  -0.1637

[112 rows x 6 columns], token_table=      Topic      Freq           Term
term                                
87        2  0.612766     absolutely
193       1  0.810503           also
100       1  0.827533         always
100       2  0.275844         always
235       2  0.905138          ample
...     ...       ...            ...
79        1  1.088337  unfortunately
35        2  0.926713           want
85        1  1.088667           work
86        1  0.845740           year
86        2  0.422870           year

[96 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2])